**Notebook para la conversión de db sqlite to pg**

Bloque 0: Inicializacion - Definición de variables

*Importacion de librerias*

In [ ]:
import sqlite3
import pandas as pd
import psycopg2
import sqlalchemy as al
from configparser import ConfigParser

*Configuraciones*

In [ ]:
# Funcion para la eExtraccion de parametros del archivo 'sqliteToPG-conf.ini' en secciones
def config(archivo='sqliteToPG-conf.ini', seccion='pg_server'):
    # Crear el parser y leer el archivo
    parser = ConfigParser()
    parser.read(archivo)
 
    # Obtener la sección de conexión a la base de datos
    result = {}
    if parser.has_section(seccion):
        params = parser.items(seccion)
        for param in params:
            result[param[0]] = param[1]
        return result
    else:
        raise Exception('Seccion {0} no encontrada en el archivo {1}'.format(seccion, archivo))

params_sqlite = config('sqliteToPG-conf.ini','sqlite') # Se extraen los parametros de la base SQlite de sqliteToPG-conf.ini
params_pg_server = config() # Se extraen los parametros del servidor postgres de sqliteToPG-conf.ini
params_newDB = config('sqliteToPG-conf.ini','pg_new_db') # Se extraen los parametros de la nueva base postgres de sqliteToPG-conf.ini
# No usar mayusculas para el nombre de la base de datos postgres

Bloque 1: Sqlite Dump Scheme and data to Pandas Dataframes

In [ ]:
# Establece la conexion con una BDD SQlite
conn = sqlite3.connect(params_sqlite['sqlite_db'])

In [ ]:
# Definicion de dataframe 'dbSchemaQueries' con sentencias SQL para la creacion del esquema de la BDD
dbTableQueries = pd.read_sql("select sql from sqlite_master", con=conn)
dbTableQueries = dbTableQueries.mask(dbTableQueries.eq(None)).dropna()

# Definicion de dataframe 'tableNames' con los nombres de las tablas
tableNames = pd.read_sql("SELECT name FROM sqlite_master WHERE (type='table')", con=conn)

# Definicion de 'dbData', lista que contendra multiples dataframes, cada uno con los datos de una tabla de la BDD
dbData = []
selectQuery = "SELECT * from "

# Iteracion en la que se carga 'dbData' con los datos de cada tabla de la BDD (un SELECT por tabla existente)
for i in range(len(tableNames)):
    currentTable = tableNames['name'][i]
    dbData.append(pd.read_sql_query(selectQuery + currentTable, con=conn))

In [ ]:
# Cierra la conexion con una BDD SQlite
conn.close()

Bloque 2: Datatypes convertion

In [ ]:
# Define una lista que almacena los valores de start para las PK de la nueva BDD
startValues = []

# Recorre dataframe a dataframe
for i in dbData:

    # Recorre las columnas del dataframe actual
    for columnName in i.columns: 

        # Cambia los datos de la columna a boolean
        if "(BL)" in columnName:
            i[columnName] = i[columnName].astype(bool)
            i.rename(columns = {columnName:columnName.replace('(BL)','')}, inplace = True)

        # Cambia los datos de la columna a datetime para su posterior almacenamiento
        """elif "date" in columnName:
            i[columnName] = pd.to_datetime(i[columnName])
            #print(i[columnName])"""

        # Busca el ultimo valor del campo id almacenado y guarda el valor siguiente, en caso de no existir id, almacena un 1
        if columnName == 'id':
            if i.empty:
                startValues.append(1)  
            else: 
                startValues.append(i.at[i.index[-1], columnName]+1)

print(startValues)

"""for i in dbData:
    for columnName in i.columns: 
        print (columnName)"""
        


Bloque 3: Scheme Parser

In [ ]:
# Definicion de funcion 'boolCHECKDelete' (limpia una Query SQL eliminando los CHECKS booleanos que pueda tener)
def boolCHECKDelete(query):

    # Define una variable auxiliar para procesar la query recibida
    parsedQuery = query

    # Define un substring con todos los CHECK de la query
    allChecks = parsedQuery[parsedQuery.index('\tCHECK'):-1]

    # Separa cada CHECK como un nuevo substring
    allChecks = allChecks.split('\n')

    # Recorre todos los CHECK que posee la query, busca si existe el tag '(BL)' y borra los que los posean (boolean CHECKS utilizados en SQlite) 
    for i in range(len(allChecks)):
        if '(BL)' in allChecks[i]:
            parsedQuery = parsedQuery.replace('\n'+allChecks[i], '')

    # Chequea si existe una ',' en el final de la query y la elimina para mantener correcta la sintaxis de la misma
    if ',' in parsedQuery[len(parsedQuery)-5:]:
        parsedQuery = parsedQuery[:len(parsedQuery)-5] + '' + parsedQuery[len(parsedQuery)-5] + '\n)'
    return parsedQuery

In [ ]:
a = dbTableQueries['sql'][4]
#a = a.replace('CREATE UNIQUE INDEX ', 'CREATE UNIQUE INDEX '+ params_newDB['new_schema_name']+'.')
#a = a.replace('ON ', 'ON '+ params_newDB['new_schema_name']+'.')
a = a.replace('CREATE TABLE ', 'CREATE TABLE '+ params_newDB['new_schema_name']+'.')
a = a.replace('REFERENCES ', 'REFERENCES '+ params_newDB['new_schema_name']+'.')
print(a)

In [ ]:
#Parseo de todas las queries de la BDD SQlite almacenadas en el dataframe 'dbSchemaQueries'

# Definicion de 'parsed_dbSchemaQueries', lista que contendra todas las queries parseadas
parsed_dbTableQueries = []
startValuesCount = 0

# Iteracion que recorre todo el dataframe, query a query.
for i in dbTableQueries.index:

    # Define una variable auxiliar para procesar la query actual
    currentQuery = dbTableQueries['sql'][i]
    
    # Reemplaza los campos DATETIME con TIMESTAMP para el pasaje a PosgreSQL
    currentQuery = currentQuery.replace('DATETIME', 'TIMESTAMP')

    # Sentencia ocasional debido al fallo en un campo de la tabla purchase, donde la longitud de los datos almacenados supera la asignada al campo
    currentQuery = currentQuery.replace('VARCHAR(16)', 'VARCHAR(32)')

    # Agrega el nombre de el Schema al que perteneceran las tablas
    currentQuery = currentQuery.replace('CREATE TABLE ', 'CREATE TABLE '+ params_newDB['new_schema_name']+'.')
    currentQuery = currentQuery.replace('REFERENCES ', 'REFERENCES '+ params_newDB['new_schema_name']+'.')

    if('CREATE UNIQUE INDEX' in currentQuery or 'CREATE INDEX' in currentQuery):
        #currentQuery = currentQuery.replace('CREATE UNIQUE INDEX ', 'CREATE UNIQUE INDEX '+ params_newDB['new_schema_name']+'.')
        currentQuery = currentQuery.replace('ON ', 'ON '+ params_newDB['new_schema_name']+'.')

    # Reemplaza los campos id para ser autoincrementales
    if 'id INTEGER NOT NULL' in currentQuery:
        # Setea el tipo de Identity, el valor de Start e incremento que tendra la tabla
        currentQuery = currentQuery.replace('id INTEGER NOT NULL,', 'id INTEGER GENERATED BY DEFAULT AS IDENTITY\n\t(START WITH ' + str(startValues[startValuesCount]) + ' INCREMENT BY 1),')
        startValuesCount = startValuesCount + 1


    # Conditions for table named "user".
    """currentQuery = currentQuery.replace('CREATE TABLE user', 'CREATE TABLE "user"')
    currentQuery = currentQuery.replace('CREATE TABLE "user"_', 'CREATE TABLE user_')
    currentQuery = currentQuery.replace('ON user', 'ON "user"')
    currentQuery = currentQuery.replace('ON "user"_', 'ON user_')
    currentQuery = currentQuery.replace('REFERENCES user', 'REFERENCES "user"')
    currentQuery = currentQuery.replace('REFERENCES "user"_', 'REFERENCES user_')"""


    # Se verifica que la query posea CHECKS, en caso de tenerlos, ejecuta 'boolCHECKDelete' en la query actual
    if(currentQuery.find('CHECK') != -1):
        currentQuery = boolCHECKDelete(currentQuery)

    # Añade la query parseada a la lista
    currentQuery = currentQuery.replace('(BL)', '')
    parsed_dbTableQueries.append(currentQuery)

# Impresion por pantalla de las queries parseadas
for i in range(len(parsed_dbTableQueries)):
    print(parsed_dbTableQueries[i])


Bloque 4: Conexión al servidor de bases de datos PostgreSQL y Creación de nueva BDD

In [ ]:
# Creamos el cursor con el objeto conexion
conexion = psycopg2.connect(**params_pg_server)
conexion.autocommit = True
cur = conexion.cursor()

# creamos la nueva BDD utilizando el cursor
createDBQuery = 'CREATE DATABASE ' + params_newDB['new_pg_db_name']
cur.execute(createDBQuery)

# crea nuevos parametros para conectarse a la BDD recien creada
#newDBparams['database'] = newPostgreSQLDbName
#newDBparams['user'] = 'gdeluca'
#newDBparams['password'] = 'gdeluca'

# Cerramos la conexión
conexion.commit()
conexion.close()

Bloque 5: Creacion del nuevo Schema

In [ ]:
# Creamos el cursor con el objeto conexion
aux_params_pg_server = params_pg_server.copy()
aux_params_pg_server['database'] = params_newDB['new_pg_db_name']
conexion = psycopg2.connect(**aux_params_pg_server)
conexion.autocommit = True
cur = conexion.cursor()

# Creamos la query para generar el nuevo Schema
createSchemaQuery = 'CREATE SCHEMA ' + params_newDB['new_schema_name'] + ';'

#Ejecutamos la query en la bdd creada
cur.execute(createSchemaQuery)

conexion.commit()
conexion.close()

Bloque 6: Creacion de las tablas en la nuevo Schema

In [ ]:
# Creamos el cursor con el objeto conexion
#aux_params_pg_server = params_pg_server.copy()
#aux_params_pg_server['database'] = params_newDB['new_pg_db_name']
conexion = psycopg2.connect(**aux_params_pg_server)
conexion.autocommit = True
cur = conexion.cursor()

# iteracion en la cual el cursor ejecuta todas las queries previamente parseadas una a una. (Crea las tablas en la nueva BDD PostgreSQL)
for i in range(len(parsed_dbTableQueries)):
    cur.execute(parsed_dbTableQueries[i])

# Cerramos la conexión
conexion.commit()
conexion.close()

Bloque 7: Carga de los datos

In [ ]:
# Creamos una conexion en 'engine' para utilizar SQLAlchemy
dbConexionString = 'postgresql://'+params_pg_server['user']+':'+params_pg_server['password']+'@'+params_pg_server['host']+':'+params_pg_server['port']+'/' + params_newDB['new_pg_db_name']
engine = al.create_engine(dbConexionString)

#for i in tableNames.index:
#    tableNames['name'][i] = tableNames['name'][i].replace(tableNames['name'][i], params_newDB['new_schema_name'] + '.' + tableNames['name'][i])

with engine.begin() as connection:
    # Iteracion donde recorre cada dataframe en 'dbData' y carga los datos formateados de cada df a la tabla correspondiente en la nueva BDD
    for i in range(len(dbData)):
        dbData[i].to_sql(tableNames['name'][i], con=connection, if_exists="append", index = False, schema=params_newDB['new_schema_name'],)
        

In [ ]:
# Cerramos el engine de SQLAlchemy
connection.close()
engine.dispose()